# Wsi zad 3

In [122]:
import random
from timeit import default_timer as timer
import pandas as pd
from IPython.display import display
initial_tokens = random.randint(8,20)
K= 3
depth = 30
state = initial_tokens
move_max = True
print(f"Number of initial tokens: {state}")


Number of initial tokens: 12


## Main algorithms

In [123]:
def heuristic_evaluation(state,K,move_max):
  if state == 0:
    return 1 if move_max else -1
  else:
      return 0

In [124]:
def mini_max_a_b(state, depth, move_max, alpha, beta,K,pruning=True):
  if depth == 0  or state == 0:
    return heuristic_evaluation(state,K,move_max)
  legal_moves = [i for i in range(1, min(state, K) + 1)]
  if move_max:
    max_eval = float('-inf')
    for move in legal_moves:
      new_state = state - move
      eval = mini_max_a_b(new_state, depth-1, not move_max, alpha, beta,K,pruning)
      max_eval = max(max_eval, eval)
      if pruning:
        alpha = max(alpha, max_eval)
        if (alpha >= beta):
          break
    return max_eval
  else:
    min_eval = float('inf')
    for move in legal_moves:
      new_state = state - move
      eval = mini_max_a_b(new_state, depth-1, not move_max, alpha, beta,K,pruning)
      min_eval = min(min_eval, eval)
      if pruning:
        beta = min(beta, min_eval)
        if (alpha >= beta):
          break
    return min_eval

In [125]:
def best_move(state, depth, move_max,K,pruning=True):
  legal_moves = [i for i in range(1, min(state, K) + 1)]
  best_moves = []
  if not legal_moves:
        return heuristic_evaluation(state,K,move_max)
  max_eval = -1
  min_eval = 1
  for move in legal_moves:
    if move_max:
        new_state = state - move
        eval = mini_max_a_b(new_state, depth, not move_max, float('-inf'), float('inf'), K,pruning)
        if eval > max_eval:
            best_moves = [move]
            max_eval = eval
        elif eval == max_eval:
            best_moves.append(move)
    else:
        new_state = state - move
        eval = mini_max_a_b(new_state, depth,not move_max, float('-inf'), float('inf'), K,pruning)
        if eval < min_eval:
            best_moves = [move]
            min_eval = eval
        elif eval == min_eval:
            best_moves.append(move)
  # print(f'Best moves: {best_moves}')
  return random.choice(best_moves)

## game simulation

In [126]:
def game(state, depth,move_max, K,pruning=True):
  tE = timer()
  move_max = True
  while state > 0:
      move = best_move(state, depth, move_max, K,pruning)
      state = state - move
      if move_max:
          # print(f"Maximizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              tQ = timer()
              return False, tQ - tE
              # print("Maximizing player loses")
      else:
          # print(f"Minimizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              tQ = timer()
              return True, tQ - tE
              # print("Minimizng player loses")
      move_max = not move_max

In [127]:
def play(state, depth,move_max, K,pruning=True):
  while state > 0:
      move = best_move(state, depth, move_max, K,pruning)
      state = state - move
      if move_max:
          print(f"Maximizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              print("Maximizing player loses")
      else:
          print(f"Minimizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              print("Minimizng player loses")
      move_max = not move_max

## Multiple games simulation

In [128]:
def percentage_of_wins(state, depth, K, move_max, iterations, pruning=True):
  wins = 0
  time = []
  for i in range(iterations):
    wins += game(state, depth, move_max,K,pruning)[0]
    time.append(game(state,depth,move_max,K,pruning)[1])
  avg_time = sum(time)/len(time)
  return f'{wins*100/iterations}', f'{round(avg_time,4)}'

# Experiments for uniform distibution

In [129]:
def game_data_uniform_distribution(start_depth, move_max, K, iter,rows, pruning=True):
    df = pd.DataFrame(columns=['range of distribution', 'K', 'depth', 'chances','average_time(sec)',"iterations"])
    depth = start_depth
    for j in range(rows):
        wins = 0
        time = []
        for i in range(iter):
            state = random.randint(8,20)
            wins += game(state, depth, move_max, K, pruning)[0]
            time.append(game(state,depth,move_max,K,pruning)[1])
        depth += 1
        avg_time = sum(time)/len(time)
        chances = f'{wins*100/iter}%'
        new_df = pd.DataFrame({'range of distribution': [f'({8},{20})'], 'K': [K], 'depth':[depth-1], 'chances': [chances],'average_time(sec)':[round(avg_time,4)],'iterations':[iter]})
        df = pd.concat([df, new_df], ignore_index=True)
    return df



Experiments with random uniform for different depths

In [132]:
df1 = game_data_uniform_distribution(0,True,3,100,20)
df3 = game_data_uniform_distribution(0,True,3,100,20,False)
# play(initial_tokens, depth, move_max,K)
display(df1,df3)

,range of distribution,K,depth,chances,average_time(sec),iterations
0,"(8,20)",3,0,55.0%,0.0000,100
1,"(8,20)",3,1,58.0%,0.0001,100
2,"(8,20)",3,2,52.0%,0.0001,100
3,"(8,20)",3,3,54.0%,0.0002,100
4,"(8,20)",3,4,61.0%,0.0004,100
5,"(8,20)",3,5,61.0%,0.0006,100
6,"(8,20)",3,6,67.0%,0.0011,100
7,"(8,20)",3,7,62.0%,0.0017,100
8,"(8,20)",3,8,72.0%,0.0032,100
9,"(8,20)",3,9,73.0%,0.0039,100


,range of distribution,K,depth,chances,average_time(sec),iterations
0,"(8,20)",3,0,46.0%,0.0000,100
1,"(8,20)",3,1,56.0%,0.0000,100
2,"(8,20)",3,2,51.0%,0.0001,100
3,"(8,20)",3,3,48.0%,0.0002,100
4,"(8,20)",3,4,54.0%,0.0006,100
5,"(8,20)",3,5,60.0%,0.0014,100
6,"(8,20)",3,6,67.0%,0.0030,100
7,"(8,20)",3,7,72.0%,0.0060,100
8,"(8,20)",3,8,69.0%,0.0097,100
9,"(8,20)",3,9,81.0%,0.0221,100


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['average_time(sec)'].plot(kind='hist', bins=20, title='average_time(sec)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='index', y='average_time(sec)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['average_time(sec)']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('average_time(sec)')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['average_time(sec)'].plot(kind='line', figsize=(8, 4), title='average_time(sec)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [135]:
df3 = game_data_uniform_distribution(0,True,3,100,20,False)
display(df3)

,range of distribution,K,depth,chances,average_time(sec),iterations
0,"(8,20)",3,0,56.0%,0.0000,100
1,"(8,20)",3,1,51.0%,0.0001,100
2,"(8,20)",3,2,40.0%,0.0001,100
3,"(8,20)",3,3,49.0%,0.0003,100
4,"(8,20)",3,4,54.0%,0.0006,100
5,"(8,20)",3,5,58.0%,0.0014,100
6,"(8,20)",3,6,58.0%,0.0031,100
7,"(8,20)",3,7,62.0%,0.0058,100
8,"(8,20)",3,8,67.0%,0.0092,100
9,"(8,20)",3,9,74.0%,0.0161,100


In [25]:
result = percentage_of_wins(18, 30, 3,True,50)
print(result)

('100.0', '0.0302')


In [26]:
def game_different_depths(start_depth, move_max, K, iter,rows, pruning=True):
    df = pd.DataFrame(columns=['Initial tokens', 'K', 'depth', 'Max player win %','average_time(sec)'])
    final_df = pd.DataFrame(columns=['Initial tokens', 'K', 'depth', 'Max player win %','average_time(sec)'])
    depth = start_depth
    state = 9;
    for k in range(3):
      for j in range(rows):
          wins = 0
          time = []
          for i in range(iter):
              wins += game(state, depth, move_max, K, pruning)[0]
              time.append(game(state,depth,move_max,K,pruning)[1])
          depth += 1
          avg_time = sum(time)/len(time)
          chances = f'{wins*100/iter}%'
          new_df = pd.DataFrame({'Initial tokens': [f'{state}'], 'K': [K], 'depth':[depth-1], 'Max player win %': [chances],'average_time(sec)':[round(avg_time,4)]})
          df = pd.concat([df, new_df], ignore_index=True)
      depth = start_depth
      state += 4
    return df

In [27]:
df2 = game_different_depths(1,True,3,50,8)
display(df2)

,Initial tokens,K,depth,Max player win %,average_time(sec)
0,9,3,1,40.0%,0.0001
1,9,3,2,30.0%,0.0001
2,9,3,3,0.0%,0.0002
3,9,3,4,0.0%,0.0003
4,9,3,5,0.0%,0.0004
5,9,3,6,0.0%,0.0004
6,9,3,7,0.0%,0.0005
7,9,3,8,0.0%,0.0005
8,13,3,1,52.0%,0.0001
9,13,3,2,50.0%,0.0002


## Simulation of best moves

In [117]:
K= 3
depth = 0
state = 2
move_max = True
pruning = True
print(f"Number of initial tokens: {state}")
while state > 0:
      move = best_move(state, depth, move_max, K,pruning)
      state = state - move
      if move_max:
          print(f"Maximizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              print("Maximizing player loses")
      else:
          print(f"Minimizing player takes: {move} tokens, remaining tokens: {state}")
          if state == 0:
              print("Minimizng player loses")
      move_max = not move_max

Number of initial tokens: 2
Best moves: [1]
Maximizing player takes: 1 tokens, remaining tokens: 1
Best moves: [1]
Minimizing player takes: 1 tokens, remaining tokens: 0
Minimizng player loses
